In [1]:
%load_ext autoreload
%autoreload 2
from dimcat import (
    Corpus,  
    Pipeline,
    IsAnnotatedFilter,
    CorpusGrouper, 
    PieceGrouper, 
    ModeGrouper, 
    ChordSymbolBigrams, 
    ChordSymbolUnigrams,
    LocalKeySlicer,
)
import pandas as pd
pd.set_option("display.max_columns", 100)

In [2]:
corpus_path = "~/romantic_piano_corpus"
corpus = Corpus(directory=corpus_path)
corpus.data

1154 files.
KEY                       -> EXTENSIONS
---------------------------------------
beethoven_piano_sonatas   -> {'.mscx': 87, '.tsv': 239}
chopin_mazurkas           -> {'.mscx': 55, '.tsv': 166}
debussy_suite_bergamasque -> {'.mscx': 4, '.tsv': 13}
dvorak_silhouettes        -> {'.mscx': 12, '.tsv': 37}
grieg_lyrical_pieces      -> {'.mscx': 66, '.tsv': 199}
liszt_pelerinage          -> {'.mscx': 19, '.tsv': 77}
medtner_tales             -> {'.mscx': 19, '.tsv': 59}
schumann_kinderszenen     -> {'.mscx': 13, '.tsv': 40}
tchaikovsky_seasons       -> {'.mscx': 12, '.tsv': 37}

None of the 287 score files have been parsed.

All 867 tabular files have been parsed, 283 of them as Annotations object(s).
KEY                       -> ANNOTATION LAYERS
----------------------------------------------
beethoven_piano_sonatas   -> staff  voice  label_type  color  
                          -> 2      1      0 (dcml)    default    20720
                          ->        2      0 (dcml)    d

In [3]:
annotated = IsAnnotatedFilter().process_data(corpus)
print(f"Before: {len(corpus.indices[()])} IDs, after filtering: {len(annotated.indices[()])}")

Before: 287 IDs, after filtering: 264


# Tokens

All symbols, independent of the local key (the mode of which changes their semantics).

In [4]:
for group, dict_of_dfs in annotated.iter_facet("expanded", concatenate=True):
    for tuple_of_all_id_tuples, all_annotations in dict_of_dfs.items():
        pass
all_annotations.sample(20)

quarterbeats duration_qb  \
                                      interval                                  
beethoven_piano_sonatas 19-1          [76.5, 78.5)          153/2         2.0   
chopin_mazurkas         BI126-4op41-3 [183.0, 186.0)          183         3.0   
medtner_tales           op14n01       [9.5, 10.0)            19/2         0.5   
grieg_lyrical_pieces    op43n05       [1.5, 2.0)              3/2         0.5   
                        op68n04       [97.0, 98.0)             97         1.0   
beethoven_piano_sonatas 01-2          [80.5, 81.0)          161/2         0.5   
                        08-3          [413.5, 415.5)        827/2         2.0   
medtner_tales           op48n02       [268.0, 271.0)          268         3.0   
chopin_mazurkas         BI126-3op41-1 [253.0, 254.0)          253         1.0   
tchaikovsky_seasons     op37a08       [81.5, 82.0)          163/2         0.5   
beethoven_piano_sonatas 19-2          [287.0, 288.0)          287         1.0   
grieg_lyrical_pieces    op57n05       [107.0, 108.0)          107         1.0   
tchaikovsky_seasons     op37a02       [167.0, 168.0)          167         1.0   
beethoven_piano_sonatas 06-3          [154.0, 154.5)          154         0.5   
                        18-3          [156.5, 157.0)        313/2         0.5   
medtner_tales           op48n02       [16.0, 19.0)             16         3.0   
beethoven_piano_sonatas 19-1          [154.0, 154.5)          154         0.5   
                        19-2          [375.5, 376.5)        751/2         1.0   
                        17-1          [270.0, 271.0)          270         1.0   
tchaikovsky_seasons     op37a02       [176.0, 177.0)          176         1.0   

                                                       mc   mn mc_onset  \
                                      interval                            
beethoven_piano_sonatas 19-1          [76.5, 78.5)     41   39        0   
chopin_mazurkas         BI126-4op41-3 [183.0, 186.0)   62   62        0   
medtner_tales           op14n01       [9.5, 10.0)       6    5      1/8   
grieg_lyrical_pieces    op43n05       [1.5, 2.0)        1    1      3/8   
                        op68n04       [97.0, 98.0)     49   49      1/4   
beethoven_piano_sonatas 01-2          [80.5, 81.0)     28   27      3/8   
                        08-3          [413.5, 415.5)  105  104        0   
medtner_tales           op48n02       [268.0, 271.0)   91   90        0   
chopin_mazurkas         BI126-3op41-1 [253.0, 254.0)   85   85      1/4   
tchaikovsky_seasons     op37a08       [81.5, 82.0)     28   27      1/2   
beethoven_piano_sonatas 19-2          [287.0, 288.0)   97   96        0   
grieg_lyrical_pieces    op57n05       [107.0, 108.0)   36   36      1/2   
tchaikovsky_seasons     op37a02       [167.0, 168.0)   84   84      1/4   
beethoven_piano_sonatas 06-3          [154.0, 154.5)   79   77      3/8   
                        18-3          [156.5, 157.0)   62   52      5/8   
medtner_tales           op48n02       [16.0, 19.0)      7    6        0   
beethoven_piano_sonatas 19-1          [154.0, 154.5)   79   77      3/8   
                        19-2          [375.5, 376.5)  126  125      3/8   
                        17-1          [270.0, 271.0)   68   68      1/2   
tchaikovsky_seasons     op37a02       [176.0, 177.0)   89   89        0   

                                                     mn_onset timesig  staff  \
                                      interval                                 
beethoven_piano_sonatas 19-1          [76.5, 78.5)          0     2/4      2   
chopin_mazurkas         BI126-4op41-3 [183.0, 186.0)        0     3/4      2   
medtner_tales           op14n01       [9.5, 10.0)         1/8     2/4      2   
grieg_lyrical_pieces    op43n05       [1.5, 2.0)          3/8     2/4      2   
                        op68n04       [97.0, 98.0)        1/4     2/4      2   
beethoven_piano_sonatas 01-2          [80.5, 81.0)        3

In [5]:
no_chord = all_annotations.root.isna()
print(f"Concatenated annotation tables contains {all_annotations.shape[0]} rows. {no_chord.sum()} of them are not chords. Their values are:")
all_annotations.label[no_chord].value_counts(dropna=False).to_dict()

Concatenated annotation tables contains 57566 rows. 677 of them are not chords. Their values are:


{'{': 640, '}': 28, '|PAC}': 6, '|HC': 2, '|PAC': 1}

In [6]:
all_chords = all_annotations[~no_chord]
print(f"Corpus contains {all_chords.shape[0]} tokens and {len(all_chords.chord.unique())} types over {len(tuple_of_all_id_tuples)} documents.")

Corpus contains 56889 tokens and 3115 types over 264 documents.


In [7]:
#from ms3 import write_tsv
#write_tsv(all_annotations[all_annotations.pedalend.notna()], './issues/pedalpoints.tsv', pre_process=False)

# Unigrams
For computing unigram statistics, the tokens need to be grouped by their occurrence within a major or a minor key because this changes their meaning. To that aim, the annotated corpus needs to be sliced into contiguous localkey segments which are then grouped into a major (`is_minor=False`) and a minor group.

In [8]:
localkey_slices = LocalKeySlicer().process_data(annotated)

In [9]:
mode_slices = ModeGrouper().process_data(localkey_slices)

## Whole dataset

In [10]:
unigrams = ChordSymbolUnigrams(once_per_group=True).process_data(mode_slices)

In [11]:
unigrams.get()

localkey_is_minor  chord        
False              I                5229
                   V7               2597
                   V                2176
                   I6               1757
                   IV                981
                                    ... 
True               I(#6)               1
                   V(642)              1
                   I(94)               1
                   It/iv               1
                   #viio2(+2)/ii       1
Name: count, Length: 3715, dtype: int64

In [12]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR UNIGRAMS
2004 types, 35610 tokens
chord
I        5229
V7       2597
V        2176
I6       1757
IV        981
V(64)     722
V43       670
V2        647
V6        638
vi        610
ii        570
ii6       548
V65       541
IV6       469
I64       445
V7(9)     363
viio6     347
V7/V      324
ii7       278
V7/IV     236
MINOR UNIGRAMS
1711 types, 21279 tokens
chord
i          2703
V          1413
V7         1173
i6          914
iv          489
V(64)       458
V6          354
VI          321
iv6         314
V43         303
i64         243
III         233
V65         222
V2          213
#viio43     209
I           208
v           170
bII         161
V7/III      158
#viio7      156


## Per corpus

In [13]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [14]:
corpus_wise_unigrams.get()

localkey_is_minor  corpus                   chord   
False              beethoven_piano_sonatas  I           2232
                                            V           1189
                                            I6          1125
                                            V7          1041
                                            V6           468
                                                        ... 
True               tchaikovsky_seasons      i(4)/v         1
                                            i/v            1
                                            ii64           1
                                            v6/iv          1
                                            ii%65(2)       1
Name: count, Length: 6705, dtype: int64

In [15]:
for (is_minor, corpus), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR unigrams (842 types, 15275 tokens)
chord
I     2232
V     1189
I6    1125
V7    1041
V6     468
chopin_mazurkas MAJOR unigrams (451 types, 5256 tokens)
chord
I       949
V7      659
V       252
IV      189
V7/V    128
debussy_suite_bergamasque MAJOR unigrams (106 types, 363 tokens)
chord
I         28
V7        22
iii       19
V7(+2)    12
IV6       12
dvorak_silhouettes MAJOR unigrams (140 types, 1067 tokens)
chord
I     254
I6    103
V7     89
V      79
IV     46
grieg_lyrical_pieces MAJOR unigrams (659 types, 4882 tokens)
chord
I     566
V7    264
V     224
IV    127
vi    116
liszt_pelerinage MAJOR unigrams (552 types, 3455 tokens)
chord
I     501
V7    250
V     155
I6    109
IV    108
medtner_tales MAJOR unigrams (636 types, 2864 tokens)
chord
I     263
V     139
V7    122
I6     83
IV     61
schumann_kinderszenen MAJOR unigrams (114 types, 714 tokens)
chord
I      106
I6      69
V       65
V7      50
V43     30
tchaikovsky_seasons MAJOR unigrams (194

## Per piece

In [16]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [17]:
piece_wise_unigrams.get()

localkey_is_minor  fname    chord 
False              01-1     V7        11
                            I6         8
                            I          7
                            ii6        5
                            V          5
                                      ..
True               op71n07  ii%65      3
                            VIM43      2
                            ii%7       2
                            V(964)     2
                            #viio2     1
Name: count, Length: 16090, dtype: int64

# Bigrams

## Whole dataset

In [18]:
bigrams = ChordSymbolBigrams(once_per_group=True).process_data(mode_slices)

In [25]:
bigrams.get()

localkey_is_minor  from           to      
False              V7             I           1243
                   V              I            572
                   I              V            431
                                  V7           386
                   V2             I6           344
                                              ... 
True               V7(#6)/V       iv7            1
                   V7(#5)/bII/iv  bII/iv         1
                   V7(#11)        V7             1
                   V7             viio43/V       1
                   vo7            i              1
Name: count, Length: 15340, dtype: int64

In [26]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in bigrams.iter():
    print(f"{modes[is_minor]} BIGRAMS\n{ugs.shape[0]} transition types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR BIGRAMS
8647 transition types, 35117 tokens
from   to 
V7     I      1243
V      I       572
I      V       431
       V7      386
V2     I6      344
V(64)  V7      310
I6     I       287
I      I6      282
V65    I       266
I      nan     254
       IV      235
V      V7      228
V43    I       214
I      V43     200
V6     I       164
IV     I       163
I      V6      148
V      I6      145
V(64)  V       142
I6     IV      139
MINOR BIGRAMS
6693 transition types, 20971 tokens
from   to 
V7     i      568
V      i      389
i      V      203
       V7     181
       i6     163
       nan    161
V(64)  V7     149
i6     i      145
V      V7     140
V(64)  V      138
V43    i      133
V65    i      129
V6     i      118
i      V6     103
V2     i6      94
i      iv      83
       V43     75
V      nan     68
V7(6)  V7      65
i      VI      64


## Per corpus

In [20]:
corpus_wise_bigrams = Pipeline([CorpusGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

In [21]:
corpus_wise_bigrams.get()

localkey_is_minor  corpus                   from    to        
False              beethoven_piano_sonatas  V7      I             528
                                            V       I             314
                                            V2      I6            238
                                            I       V             227
                                            V(64)   V7            213
                                                                 ... 
True               tchaikovsky_seasons      V       V43(4)/V        1
                                            i       #viio43(2)      1
                                            V2/bII  ii%65(2)        1
                                            i       V2              1
                                            V65     ii%43           1
Name: count, Length: 19656, dtype: int64

In [22]:
for (is_minor, corpus), ugs in corpus_wise_bigrams.iter():
    print(f"{corpus} {modes[is_minor]} bigrams ({ugs.shape[0]} transition types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR bigrams (3573 transition types, 15226 tokens)
from   to
V7     I     528
V      I     314
V2     I6    238
I      V     227
V(64)  V7    213
chopin_mazurkas MAJOR bigrams (1418 transition types, 5202 tokens)
from  to
V7    I     337
I     IV     94
      V7     86
      V      67
IV    I      66
debussy_suite_bergamasque MAJOR bigrams (245 transition types, 354 tokens)
from  to   
I     iii      8
V7    I        7
iii6  V7       5
iii   bIII6    5
      V64      4
dvorak_silhouettes MAJOR bigrams (355 transition types, 1020 tokens)
from  to
V7    I     63
I     V7    45
V     I     36
V2    I6    29
I     I6    20
grieg_lyrical_pieces MAJOR bigrams (1755 transition types, 4803 tokens)
from   to
V7     I     99
V      I     36
I      V     34
V7(9)  I     31
I(9)   I     30
liszt_pelerinage MAJOR bigrams (1485 transition types, 3346 tokens)
from  to
V7    I     96
V     I     47
I     V     43
      V7    41
I(9)  I     41
medtner_tales MAJOR bigrams (1608 

## Per piece

In [23]:
piece_wise_bigrams = Pipeline([PieceGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

('beethoven_piano_sonatas', '23-2', Interval(190.0, 194.0, closed='left')): DataFrame has only one row, cannot compute bigram.
Group '(True, '23-2')' will be missing from the processed data.


In [24]:
piece_wise_bigrams.get()

localkey_is_minor  fname    from     to    
False              01-1     V(64)    V7        4
                            viio7/V  V(64)     3
                            ii6(2)   ii6       3
                            V7       I         3
                            V65/V    V         3
                                              ..
True               op71n07  V(964)   ii%65     2
                            VIM43    ii%7      2
                            ii%65    VIM43     2
                            ii%7     V(964)    2
                            ii%65    #viio2    1
Name: count, Length: 28051, dtype: int64